** Neil Garrett, June 2018 **

# Start up commands/load relevant functions

In [1]:
# load required libraries
using Distributed

# # set everything up
parallel = true # Run on multiple CPUs. If youhttp://localhost:8888/notebooks/Dropbox/Daw_Lab/PreySelection/v103/models/model_subjective1beta2lr_delayreward/model_subjective1beta2lr_delayreward.jl.ipynb# are having trouble, set parallel = false: easier to debug

# this activates the multiprocessing threads
if (parallel)
	# only run this once
    addprocs(4)
end

# load required libraries
@everywhere using DataFrames
#using DataArrays
@everywhere using ForwardDiff
@everywhere using PyCall
@everywhere using Distributions
@everywhere using PyPlot
@everywhere using CSV
@everywhere using SpecialFunctions
@everywhere using SharedArrays
@everywhere using LinearAlgebra

@everywhere PyCall.@pyimport scipy.optimize as so

# this is the code for the actual fitting routines
@everywhere include("em.jl")
@everywhere include("common.jl")
@everywhere include("likfuns.jl")

# this is generates starting matricies for betas, sigmas etc to feed into model
@everywhere include("genVars.jl")

┌ Warning: Deprecated syntax `(covvar < 0) ? NaN:` at /Users/neil/GitHubRepo/Projects/PreySelection/v105/models/supplementary/model_MVT_learn_options/em.jl:288.
│ Use `(covvar < 0) ? NaN :` instead.
└ @ nothing /Users/neil/GitHubRepo/Projects/PreySelection/v105/models/supplementary/model_MVT_learn_options/em.jl:288
┌ Warning: Deprecated syntax `(covvar < 0) ? NaN :sqrt` at /Users/neil/GitHubRepo/Projects/PreySelection/v105/models/supplementary/model_MVT_learn_options/em.jl:288.
│ Use `(covvar < 0) ? NaN : sqrt` instead.
└ @ nothing /Users/neil/GitHubRepo/Projects/PreySelection/v105/models/supplementary/model_MVT_learn_options/em.jl:288
┌ Warning: Deprecated syntax `(diag(covmtx)[i] .< 0) ? NaN:` at /Users/neil/GitHubRepo/Projects/PreySelection/v105/models/supplementary/model_MVT_learn_options/em.jl:299.
│ Use `(diag(covmtx)[i] .< 0) ? NaN :` instead.
└ @ nothing /Users/neil/GitHubRepo/Projects/PreySelection/v105/models/supplementary/model_MVT_learn_options/em.jl:299
┌ Warning: Deprecat

┌ Warning: Deprecated syntax `parametric method syntax flatten{T}(a::Array{T})` around /Users/neil/GitHubRepo/Projects/PreySelection/v105/models/supplementary/model_MVT_learn_options/common.jl:67.
│ Use `flatten(a::Array{T}) where T` instead.
└ @ ~/GitHubRepo/Projects/PreySelection/v105/models/supplementary/model_MVT_learn_options/common.jl:67
┌ Warning: Deprecated syntax `any(x -> begin
│     # /Users/neil/GitHubRepo/Projects/PreySelection/v105/models/supplementary/model_MVT_learn_options/common.jl, line 66
│     isa(x, Array)
│ end, a)?` at /Users/neil/GitHubRepo/Projects/PreySelection/v105/models/supplementary/model_MVT_learn_options/common.jl:66.
│ Use `any(x -> begin
│     # /Users/neil/GitHubRepo/Projects/PreySelection/v105/models/supplementary/model_MVT_learn_options/common.jl, line 66
│     isa(x, Array)
│ end, a) ?` instead.
└ @ ~/GitHubRepo/Projects/PreySelection/v105/models/supplementary/model_MVT_learn_options/common.jl:66
┌ Warning: Deprecated syntax `any(x -> begin
│     

# Data read and process

### Read in trial by trial data

In [2]:
#read in csv file of the data
#trial by trial data: note will include force trials and missed responses
df = readtable("/Users/neil/GitHubRepo/Projects/PreySelection/v105/data/trialdata_105_processed.csv")

#display header
head(df)


┌ Warning: readtable is deprecated, use CSV.read from the CSV package instead
│   caller = top-level scope at In[2]:1
└ @ Core In[2]:1


,subj,trial_index_actual,block,stimulus,stim_rank,reward_percent,delay_s,profitability,stim_left_right,key_press,approach_avoid,rt,rt_z,force_trial,missed,order_condition,exclude,exclude_reason
,Int64⍰,Int64⍰,Int64⍰,String⍰,Int64⍰,Int64⍰,Int64⍰,Float64⍰,String⍰,Int64⍰,Float64⍰,Float64⍰,Float64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,String⍰
1,1,0,0,../static/images/invador1.png,4,20,8,2.5,right,74,1.0,1195.0,1.09567,1,0,1,0,do not exclude
2,1,1,0,../static/images/invador3.png,1,80,2,40.0,right,74,1.0,718.0,-1.04464,0,0,1,0,do not exclude
3,1,2,0,../static/images/invador2.png,2,20,2,10.0,left,70,NaN,NaN,NaN,2,1,1,0,do not exclude
4,1,3,0,../static/images/invador4.png,3,80,8,10.0,left,70,1.0,933.0,-0.0799261,0,0,1,0,do not exclude
5,1,4,0,../static/images/invador3.png,1,80,2,40.0,left,70,1.0,750.0,-0.901051,1,0,1,0,do not exclude
6,1,5,0,../static/images/invador3.png,1,80,2,40.0,right,74,1.0,628.0,-1.44847,0,0,1,0,do not exclude


### Append data with the column "sub" 


In [3]:
#this is just a replica of the existing column sub_no but think em looks for "sub" specifically
df[:sub] = df[:subj];


### Get rid of excluded subs

In [4]:
df = df[df[:exclude].==0,:];


### Convert approach avoid to 2s and 1s , missed as 0. Then convert to integers (necessary to use as an index)

In [5]:
#convert approach_avoid to 1s (avoid) and 2s (approach)
df[df[:approach_avoid].==1,:approach_avoid] = 2
df[df[:approach_avoid].==-1,:approach_avoid] = 1

index_NaN = find(isnan.(df[:approach_avoid]))
df[index_NaN, :approach_avoid] = 0

df[:approach_avoid] = convert(Vector{Integer}, df[:approach_avoid])

head(df)


┌ Warning: `find(A::AbstractVector)` is deprecated, use `findall(A)` instead.
│   caller = top-level scope at In[5]:4
└ @ Core In[5]:4


,subj,trial_index_actual,block,stimulus,stim_rank,reward_percent,delay_s,profitability,stim_left_right,key_press,approach_avoid,rt,rt_z,force_trial,missed,order_condition,exclude,exclude_reason,sub
,Int64⍰,Int64⍰,Int64⍰,String⍰,Int64⍰,Int64⍰,Int64⍰,Float64⍰,String⍰,Int64⍰,Integer,Float64⍰,Float64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,String⍰,Int64⍰
1,1,0,0,../static/images/invador1.png,4,20,8,2.5,right,74,2,1195.0,1.09567,1,0,1,0,do not exclude,1
2,1,1,0,../static/images/invador3.png,1,80,2,40.0,right,74,2,718.0,-1.04464,0,0,1,0,do not exclude,1
3,1,2,0,../static/images/invador2.png,2,20,2,10.0,left,70,0,NaN,NaN,2,1,1,0,do not exclude,1
4,1,3,0,../static/images/invador4.png,3,80,8,10.0,left,70,2,933.0,-0.0799261,0,0,1,0,do not exclude,1
5,1,4,0,../static/images/invador3.png,1,80,2,40.0,left,70,2,750.0,-0.901051,1,0,1,0,do not exclude,1
6,1,5,0,../static/images/invador3.png,1,80,2,40.0,right,74,2,628.0,-1.44847,0,0,1,0,do not exclude,1


### Read in summary stats

In [6]:
summary_stats = readtable("/Users/neil/GitHubRepo/Projects/PreySelection/v105/data/subdata_105.csv")
head(summary_stats)

┌ Warning: readtable is deprecated, use CSV.read from the CSV package instead
│   caller = top-level scope at In[6]:1
└ @ Core In[6]:1


,MturkID,AssignID,sub_no,age,gender,n_trials,n_force_wrong,n_missed,bonus_payment,percent_accept_A1,percent_accept_A2,percent_accept_A3,percent_accept_A4,percent_accept_A2_A3,percent_accept_B1,percent_accept_B2,percent_accept_B3,percent_accept_B4,percent_accept_B2_B3,percent_accept_B1_min_A1,percent_accept_B2_min_A2,percent_accept_B3_min_A3,percent_accept_B4_min_A4,percent_accept_B2_B3_min_A2_A3,percent_accept_AB1,percent_accept_AB2,percent_accept_AB3,percent_accept_AB4,percent_accept_AB2_AB3,exclude,exclude_reason,order_condition,comment
,String⍰,String⍰,Int64⍰,Int64⍰,String⍰,Int64⍰,Int64⍰,Int64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Int64⍰,String⍰,Int64⍰,String⍰
1,A3VE8U98QF4FVV,3RSDURM96AM9RPFMDMRKDJZ2IHMEYS,1,30,m,284,1,1,2.7,1.0,0.0714286,0.117647,0.0,0.0967742,1.0,0.857143,1.0,0.0227273,0.9375,0.0,0.785714,0.882353,0.0227273,0.840726,1.0,0.333333,0.423077,0.225,0.382979,0,do not exclude,1,Everything went fine and I didn't have any problems and the game was fun.
2,A2LI0QVPKA5CG8,3HQUKB7LNFEZP4GZTNV2CM1CN34HHM,2,40,f,255,1,1,2.4,1.0,0.0,1.0,0.111111,0.45,0.785714,0.166667,1.0,0.0377358,0.583333,-0.214286,0.166667,0.0,-0.0733753,0.133333,0.942308,0.0869565,1.0,0.108434,0.522727,0,do not exclude,2,A great game that was extremely fun! Thank you!
3,AMYZ8WZD19W29,39ZSFO5CA8W0LWH9HM5M7E6HU3AJUU,3,30,m,333,2,4,2.8,1.0,0.235294,0.117647,0.05,0.176471,1.0,0.733333,0.647059,0.0,0.6875,0.0,0.498039,0.529412,-0.05,0.511029,1.0,0.46875,0.382353,0.190476,0.424242,0,do not exclude,1,I feel like the red asterisk was there to annoy me.
4,AYNWXFK66TKOZ,3TPWUS5F891Q5M082U312YEJPZCCW8,4,29,f,350,5,8,2.8,0.987013,0.95,0.0526316,0.0,0.512821,0.933333,0.944444,0.8,0.0166667,0.878788,-0.0536797,-0.00555556,0.747368,0.0166667,0.365967,0.978261,0.947368,0.382353,0.178218,0.680556,0,do not exclude,2,"Overall a fun activity, curious about what the study is about."
5,A1HKCTDOTL6C9I,37XITHEISW9LXQJM30TVFZ62XKXCR5,5,42,m,301,0,4,2.8,1.0,0.428571,0.882353,0.25,0.677419,0.933333,0.785714,0.933333,0.0172414,0.862069,-0.0666667,0.357143,0.0509804,-0.232759,0.18465,0.986486,0.607143,0.90625,0.244681,0.766667,0,do not exclude,1,Thank you.
6,A3T9C1USL4V4IJ,3FUI0JHJPXY51O21ZHHY4YZSLT533V,6,36,male,321,2,12,2.7,0.985507,0.0714286,0.0,0.0,0.03125,1.0,0.4,0.785714,0.016129,0.586207,0.0144928,0.328571,0.785714,0.016129,0.554957,0.988095,0.241379,0.34375,0.158416,0.295082,0,do not exclude,2,missing


### Get rid of excluded subs

In [7]:
summary_stats = summary_stats[summary_stats[:exclude].==0,:];


### Get rid of Mturk ID etc (first 3 columns)

In [8]:
summary_stats = summary_stats[:,4:end];

┌ Warning: indexing with colon as row will create a copy in the future use df[col_inds] to get the columns without copying
│   caller = top-level scope at In[8]:1
└ @ Core In[8]:1


# MVT Model Learn Options

This model comprises: 

1. An intercept which reflects degree of bias to reject.

2. A beta (termperature parameter) which controls sensitivity to the difference between the options (0 = pick 50/50. Higher it is, the more sensative subs are tothe different options (more step functionesque). <br>

3. One learning rate for average reward rate

4. Seperate learning rate for learning the value of the options (no longer assume they know this)

Uses Q learned average to predict choice

Initalise Qaverage in model at the arithmetic average over all subs over both sessions

In [9]:
@everywhere function model_MVT_learn_options(params, data)
     
    #model parameters
    intercept = params[1]
    beta = params[2]
    lr_environment = 0.5 .+ 0.5.*erf(params[3]/sqrt(2))
    lr_options = 0.5 .+ 0.5.*erf(params[4]/sqrt(2))
    
    #initalise various variables
    delay_sum = zeros(typeof(beta),1)
    reward_sum = zeros(typeof(beta),1)
    Q_arithmetic = zeros(typeof(beta),1) # stores actual (arithmetic) average reward rate
    opp_cost_arithmetic = zeros(typeof(beta),1) # stores actual (arithmetic) opp cost
    Q_estimate = zeros(typeof(beta),1) .+ 7.02 # stores estimated global reward rate
    opp_cost_estimate = zeros(typeof(beta),1) # stores estimated opp cost
    
    Qd = zeros(typeof(beta),2) # decision variable; 1st element is the opp cost of accepting (or value of rejecting), 2nd element is just the reward of the option (value of accepting)

    # maintains estimate of the reward for each option (options indexed by rank: 1-4). 
    Q_options_reward = zeros(typeof(beta),4)
    Q_options_delay = zeros(typeof(beta),4)
    
    lik = 0 #likelihood

    #these store new trial by trial values (e.g. Q estimate on each trial etc.)
    reward_sum_store = [];
    delay_sum_store = [];
    Q_arithmetic_store = [];
    opp_cost_arithmetic_store  = []; 
    Q_estimate_store = [];
    opp_cost_estimate_store = [];

    #extract various variables from the dataframe
    reward = data[:reward_percent]
    delay = data[:delay_s]
    force = data[:force_trial]  
    missed = data[:missed] #missed responses 
    c = data[:approach_avoid] #choice
    option_rank = data[:stim_rank] #option rank 

    for i = 1:length(c)
        
            #option presented in current trial
            option_index = option_rank[i]
        
            # 2 seconds without reward on each trial regadless of accept/reject
            delay_sum .+= 2;
         
            # calculate current (arithmetic) reward per second from number of seconds elapsed and reward accured
            Q_arithmetic = reward_sum./delay_sum
            opp_cost_arithmetic = Q_arithmetic*delay[i]
           
            # decrease estimate of global reward rate for encounter time (2seconds)
            Q_estimate = (1-lr_environment) * Q_estimate .+ 0
            Q_estimate = (1-lr_environment) * Q_estimate .+ 0
        
            #calculate estimate of opportunity cost given estimate of reward rate and delay incurred by option 
            opp_cost_estimate = Q_estimate*Q_options_delay[option_index]
        
            #add trial by trial values 
            append!(reward_sum_store, reward_sum)
            append!(delay_sum_store, delay_sum)
            append!(Q_arithmetic_store, Q_arithmetic)
            append!(opp_cost_arithmetic_store, opp_cost_arithmetic)
            append!(Q_estimate_store, Q_estimate)
            append!(opp_cost_estimate_store, opp_cost_estimate)
        
            # if not a force trial predict choice based on current values
            if ((force[i]<1) & (missed[i]<1))
                        
                # decision variable - the estimate of opportunity cost ("reward" of rejecting) versus 
                # reward of the current option (if accepted)
                Qd = [intercept, 0] .+ [beta.*opp_cost_estimate[1], beta.*Q_options_reward[option_index]]

                # increment likelihood
                lik += Qd[c[i]] - log(sum(exp.(Qd)))
            
            end
            
            #incur 8second time out for missed response
            if (missed[i]==1)
                
                delay_sum .+= 8
            
                for j = 1:8
                
                     Q_estimate = (1-lr_environment) * Q_estimate .+ 0

                end
            
            end
        
            # regardless of whether a force trial or not, 
            # if accept the option, Q_estimate updates and there is a delay incurred
            if ((c[i] == 2) & (missed[i]==0))
                
                delay_sum .+= delay[i]
                reward_sum .+= reward[i]
            
                for j = 1:delay[i]
                
                    Q_estimate = (1-lr_environment) * Q_estimate .+ 0
                
                end
            
                    Q_estimate = (1-lr_environment) * Q_estimate .+ lr_environment*reward[i]
            
                Q_options_reward[option_index] = (1-lr_options)*Q_options_reward[option_index] .+ lr_options*reward[i]
                Q_options_delay[option_index] = (1-lr_options)*Q_options_delay[option_index] .+ lr_options*delay[i]
                
            end
    
    end
    
    # compile trial by trial values here
    trial_data = DataFrame(reward_sum = reward_sum_store,
            delay_sum = delay_sum_store,
            Q_arithmetic = Q_arithmetic_store,
            opp_cost_arithmetic = opp_cost_arithmetic_store,
            Q_estimate = Q_estimate_store,
            opp_cost_estimate = opp_cost_estimate_store)
    
    # here if running em you can only return the likelihood
    return -lik
    
    # but if you run in order to extract trials, subs etc then want to return this
    #return (-lik, trial_data)
    
end


# Parameter optimisiation

### Run model for one subject

aids debugging

In [10]:
# initialize parameter structures
(df, subs, X, betas, sigma) = genVars(df, 4);

# run model for sub 1
model_MVT_learn_options(betas,df[df[:sub].==subs[1],:])


140.0157304731092

### Run em to get best fit parameters for each subject

In [11]:
# initialized parameter structures (again)
# note that some of the variables (e.g. betas, sigma) are entered and returned by em function 
(df, subs, X, betas, sigma) = genVars(df, 4);

# run for full learner
# x contains the parameters for each subject (note not the same as variable X)
# l and h are per-subject likelihood and hessians
@time (betas, sigma, x, l, h) = em(df, subs, X, betas, sigma, model_MVT_learn_options; emtol=1e-3, parallel=true, full=true, quiet=false);



iter: 25
betas: [0.65, 0.11, -1.21, -0.33]
sigma: [0.64 -0.01 0.18 -0.13; -0.01 0.0 -0.02 -0.03; 0.18 -0.02 1.87 -0.24; -0.13 -0.03 -0.24 1.06]
change: [7.2e-5, 0.000114, -1.9e-5, -0.000315, 8.4e-5, -0.000143, 0.000381, -0.000171, 0.000884, -0.00027, -0.000238, 3.2e-5, -0.000767, 5.7e-5]
max: 0.000884
391.811952 seconds (33.50 M allocations: 1.389 GiB, 0.83% gc time)


### Generate Model Statistics 
(IAIC, IBIC, LOOCV)

In [12]:
## model selection/comparison/scoring

# laplace approximation to the aggregate log marginal likelihood of the whole dataset
# marginalized over the individual params

aggll = lml(x,l,h)

# to compare this between models you need to correct for the group-level free parameters
# either aic or bic

aggll_ibic = ibic(x,l,h,betas,sigma,nrow(df))
aggll_iaic = iaic(x,l,h,betas,sigma)

# or you can compute unbiased per subject marginal likelihoods via subject-level cross validation
# you can do paired t tests on these between models
# these are also appropriate for SPM_BMS etc

# takes ages so comment in when want to run, otherwise just use IAIC above
liks = loocv(df, subs, x, X, betas, sigma, model_MVT_learn_options; emtol=1e-3, parallel=true, full=true)
#aggll_loo = sum(liks)

#println("\n\nraw nll:  $aggll\nibic nll: $aggll_ibic\niaic nll: $aggll_iaic\nloo nll:  $aggll_loo")
#println("\n\nraw nll:  $aggll\nibic nll: $aggll_ibic\niaic nll:")
#print(aggll_iaic)


Subject: 1..2..3..4..5..6..7..8..9..10..11..12..13..14..15..16..17..18..19..20..21..22..23..24..25..26..27..28..29..30..31..32..33..34..35..36..37..38..

38-element Array{Float64,1}:
  51.578514191111026
  63.18311515949695 
  80.89587235437733 
 101.24569659255911 
  95.03929125072781 
  67.47377339063027 
  61.8412083956431  
  62.87575103582741 
  66.16103276599306 
 100.91662460025385 
  99.5518092801466  
  84.3301797442848  
  93.3261276544475  
   ⋮                
  63.09694237049189 
  67.11243028273995 
  93.5313668541738  
  88.35164841412899 
  52.70120265956687 
 115.6598516433405  
  61.497198975623206
  77.97599240128831 
  52.325946653334334
  35.9386798781354  
  91.74661681543526 
  98.43971536644443 

### Write loocv scores to csv file

(if you have run loocv above)

In [13]:
# put loocv scores into dataframe
loocv_scores = DataFrame(sub = subs,
liks = vec(liks));


#### save LOOCV to csv file


In [14]:
CSV.write("loocv_scores.csv", DataFrame(loocv_scores))


┌ Warning: In the future DataFrame constructor called with a `DataFrame` argument will return a copy. Use `convert(DataFrame, df)` to avoid copying if `df` is a `DataFrame`.
│   caller = top-level scope at In[14]:1
└ @ Core In[14]:1


"loocv_scores.csv"

#### add to summary stats to LOOCV as well

In [15]:
summary_stats = [summary_stats loocv_scores];

### Calculate and write p values, std error and covariance

In [16]:
# standard errors on the subject-level means, based on an asymptotic Gaussian approx 
# (these may be inflated esp for small n)
(standarderrors, pvalues, covmtx) = emerrors(df,subs,x,X,h,betas,sigma,model_MVT_learn_options);


  likely near /Users/neil/.julia/packages/IJulia/DL02A/src/kernel.jl:41
  likely near /Users/neil/.julia/packages/IJulia/DL02A/src/kernel.jl:41
  likely near /Users/neil/.julia/packages/IJulia/DL02A/src/kernel.jl:41
in #53 at none
  likely near /Users/neil/.julia/packages/IJulia/DL02A/src/kernel.jl:41
  likely near /Users/neil/.julia/packages/IJulia/DL02A/src/kernel.jl:41
  likely near /Users/neil/.julia/packages/IJulia/DL02A/src/kernel.jl:41
in #53 at none
┌ Warning: `ccdf(d::UnivariateDistribution, X::AbstractArray)` is deprecated, use `ccdf.(d, X)` instead.
│   caller = emerrors(::DataFrame, ::Array{Union{Missing, Int64},1}, ::SharedArray{Float64,2}, ::Array{Float64,3}, ::SharedArray{Float64,3}, ::Array{Float64,1}, ::Array{Float64,2}, ::Function) at em.jl:300
└ @ Main /Users/neil/GitHubRepo/Projects/PreySelection/v105/models/supplementary/model_MVT_learn_options/em.jl:300


In [17]:
model_stats = DataFrame(stderror = vec(standarderrors),
pvalues = vec(pvalues),
covmtx_1 = vec(covmtx[:,1]),
covmtx_2 = vec(covmtx[:,2]),
covmtx_3 = vec(covmtx[:,3]),
covmtx_4 = vec(covmtx[:,4]));

# save model stats to csv file
CSV.write("model_stats.csv", DataFrame(model_stats));

┌ Warning: In the future DataFrame constructor called with a `DataFrame` argument will return a copy. Use `convert(DataFrame, df)` to avoid copying if `df` is a `DataFrame`.
│   caller = top-level scope at In[17]:7
└ @ Core In[17]:7


In [18]:
print(standarderrors)


[0.137452, 0.0118718, 0.223839, 0.226929]

In [19]:
print(pvalues)


[2.00286e-6, 6.94856e-22, 6.15916e-8, 0.146612]

In [20]:
print(covmtx)


[0.018893 -0.000250757 0.00510282 -0.00350653; -0.000250757 0.00014094 -0.0005697 -0.000852077; 0.00510282 -0.0005697 0.0501037 -0.00662212; -0.00350653 -0.000852077 -0.00662212 0.0514968]

### Write per subject model parameters to csv files

In [21]:
# put parameters into variable d
d=x';

# now put parameters into dataframe
params = DataFrame(sub = subs,
intercept = vec(d[:,1]), 
beta = vec(d[:,2]),
learning_rate_environment_raw = vec(d[:,3]),
learning_rate_environment_transformed = vec(0.5 .+ 0.5*erf.(d[:,3] / sqrt(2))),
learning_rate_options_raw = vec(d[:,4]),
learning_rate_options_transformed = vec(0.5 .+ 0.5*erf.(d[:,4] / sqrt(2))));


#### save parameters to csv file


In [22]:
CSV.write("subject_params.csv", DataFrame(params))


┌ Warning: In the future DataFrame constructor called with a `DataFrame` argument will return a copy. Use `convert(DataFrame, df)` to avoid copying if `df` is a `DataFrame`.
│   caller = top-level scope at In[22]:1
└ @ Core In[22]:1


"subject_params.csv"